# MLflow Regression Pipeline Notebook

This notebook runs the MLflow Regression Pipeline on Databricks and inspects its results. For more information about the MLflow Regression Pipeline, including usage examples, see the [Regression Pipeline overview documentation](https://mlflow.org/docs/latest/pipelines.html#regression-pipeline) the [Regression Pipeline API documentation](https://mlflow.org/docs/latest/python_api/mlflow.pipelines.html#module-mlflow.pipelines.regression.v1.pipeline).

In [79]:
!C:\Users\GEBETSBE\AppData\Local\Programs\Python\Python39\Scripts\mlflow --version

mlflow, version 1.28.0


In [80]:
from mlflow.pipelines import Pipeline
import os

chocoportable_path = "C:\\ProgramData\\chocoportable\\bin\\"
if chocoportable_path not in os.environ['PATH']:
    os.environ['PATH'] += ";" + chocoportable_path

p = Pipeline(profile="local")

2022/08/16 14:27:33 INFO mlflow.pipelines.pipeline: Creating MLflow Pipeline 'mlp-regression-template' with profile: 'local'


In [81]:
p.clean()

In [82]:
p.inspect()

In [83]:
p.run("ingest")

2022/08/16 14:27:36 INFO mlflow.pipelines.steps.ingest.datasets: Resolving input data from '['C:\\Users\\GEBETSBE\\Documents\\Dev\\mlp-regression-template\\data\\combined_disasters.csv']'
2022/08/16 14:27:36 INFO mlflow.pipelines.steps.ingest.datasets: Resolved input data to 'C:\Users\GEBETSBE\AppData\Local\Temp\tmpskp_fp7n\combined_disasters.csv'
2022/08/16 14:27:36 INFO mlflow.pipelines.steps.ingest.datasets: Converting dataset to parquet format, if necessary
Loading dataset CSV using `pandas.read_csv()` with default arguments and assumed index column 0 which may not produce the desired schema. If the schema is not correct, you can adjust it by modifying the `load_file_as_dataframe()` function in `steps/ingest.py`
2022/08/16 14:27:43 INFO mlflow.pipelines.steps.ingest: Successfully stored data in parquet format at 'C:\Users\GEBETSBE\.mlflow\pipelines\202617ce1eed864f31ad8970a0482c9fc9fca989f030c04989a44ab6a7a1c30d\steps\ingest\outputs\dataset.parquet'
2022/08/16 14:27:43 INFO mlflow.

In [84]:
p.run("split")

2022/08/16 14:27:51 INFO mlflow.pipelines.steps.split: Creating hash buckets on input dataset containing 94332 rows consumes 0.2647840976715088 seconds.


In [85]:
p.run("train")

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
2022/08/16 14:28:01 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
C:\Users\GEBETSBE\Documents\Dev\mlp-regression-template\venv\lib\site-packages\_distutils_hack\__init__.py:36: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
C:\Users\GEBETSBE\Documents\Dev\mlp-regression-template\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
2022/08/16 14:28:10 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2022/08/16 14:28:10 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
C:\Users\GEBETSBE\Documents\Dev\mlp-regression-template\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression 

Metric,training,validation
root_mean_squared_error,15.8914,15.959
example_count,75697,9459
max_error,104.059,104.059
mean_absolute_error,12.1673,12.2424
mean_absolute_percentage_error,0.00609623,0.00613359
mean_on_label,2002.28,2002.32
mean_squared_error,252.536,254.688
r2_score,0.015397,0.0161557
score,0.015397,0.0161557
sum_on_label,1.51566e+08,1.89399e+07


In [86]:
p.run("evaluate")

2022/08/16 14:28:15 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
C:\Users\GEBETSBE\Documents\Dev\mlp-regression-template\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\GEBETSBE\Documents\Dev\mlp-regression-template\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
X does not have valid feature names, but LinearRegression was fitted with feature names
2022/08/16 14:28:16 INFO mlflow.models.evaluation.default_evaluator: Shap explainer _PatchedKernelExplainer is used.

  0%|          | 0/10 [00:00<?, ?it/s]X does not have valid feature names, but LinearRegression was fitted with feature names
X does not have valid feature names, but LinearRegression was fitted with feature names
X does not have valid feature names, bu

Metric,validation,test
root_mean_squared_error,15.959,16.249686
example_count,9459,9176.000000
max_error,104.059,103.199975
mean_absolute_error,12.2424,12.308155
mean_absolute_percentage_error,0.00613359,0.006169
mean_on_label,2002.32,2002.201395
mean_squared_error,254.688,264.052280
r2_score,0.0161557,0.017194
score,0.0161557,0.017194
sum_on_label,1.89399e+07,18372200.000000


In [87]:
p.run("register")

In [88]:
p.inspect("train")

Metric,training,validation
root_mean_squared_error,15.8914,15.959
example_count,75697,9459
max_error,104.059,104.059
mean_absolute_error,12.1673,12.2424
mean_absolute_percentage_error,0.00609623,0.00613359
mean_on_label,2002.28,2002.32
mean_squared_error,252.536,254.688
r2_score,0.015397,0.0161557
score,0.015397,0.0161557
sum_on_label,1.51566e+08,1.89399e+07


In [89]:
test_data = p.get_artifact("test_data")
test_data.describe()

,roughYear
count,9176.000000
mean,2002.201395
std,16.392102
min,1900.000000
25%,2000.000000
50%,2000.000000
75%,2020.000000
max,2020.000000


In [90]:
trained_model = p.get_artifact("model")
print(trained_model)

mlflow.pyfunc.loaded_model:
  artifact_path: train/model
  flavor: mlflow.sklearn
  run_id: ba0af852624e4f39859d4fea5187ac44

